# Other Example and Exercises in Bag

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. For this pourposes many Distributed computation frameworks use the *functional* programming style fits very nicely with standard Python iteration (as well R, Scala and Java 8), such as can be found in the `itertools` module.

Messy data are very common when you in the life of a Data scientist and they are often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s. Working with Dask DataFrame or Pandas is not a good idea until the data have been normalized.

Python collection data structures are optimized for general-purpose storage and processing. If we combine this property with the parallel/distributed computation then we can process a large amount of data.


In [1]:
from dask.distributed import Client

client = Client()

## Let's see some examples

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [2]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [3]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data','accounts_json','accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Dan", "transactions": [{"transaction-id": 14, "amount": 4526}, {"transaction-id": 2814, "amount": 4412}, {"transaction-id": 4918, "amount": 4301}, {"transaction-id": 12573, "amount": 4215}, {"transaction-id": 14264, "amount": 4428}, {"transaction-id": 15537, "amount": 4317}, {"transaction-id": 18212, "amount": 4644}, {"transaction-id": 21465, "amount": 4351}, {"transaction-id": 24422, "amount": 4292}, {"transaction-id": 29525, "amount": 4297}, {"transaction-id": 36043, "amount": 4232}, {"transaction-id": 41620, "amount": 4405}, {"transaction-id": 42443, "amount": 4356}, {"transaction-id": 43028, "amount": 4243}, {"transaction-id": 46824, "amount": 4320}, {"transaction-id": 48087, "amount": 4501}, {"transaction-id": 51930, "amount": 4345}, {"transaction-id": 52633, "amount": 4214}, {"transaction-id": 57717, "amount": 4388}, {"transaction-id": 60042, "amount": 4419}, {"transaction-id": 63538, "amount": 4434}, {"transaction-id": 70414, "amount": 4357}, {"transaction-i

In [4]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [5]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [6]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [7]:
filename = os.path.join('data', 'accounts_json', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Dan", "transactions": [{"transaction-id": 14, "amount": 4526}, {"transaction-id": 2814, "amount": 4412}, {"transaction-id": 4918, "amount": 4301}, {"transaction-id": 12573, "amount": 4215}, {"transaction-id": 14264, "amount": 4428}, {"transaction-id": 15537, "amount": 4317}, {"transaction-id": 18212, "amount": 4644}, {"transaction-id": 21465, "amount": 4351}, {"transaction-id": 24422, "amount": 4292}, {"transaction-id": 29525, "amount": 4297}, {"transaction-id": 36043, "amount": 4232}, {"transaction-id": 41620, "amount": 4405}, {"transaction-id": 42443, "amount": 4356}, {"transaction-id": 43028, "amount": 4243}, {"transaction-id": 46824, "amount": 4320}, {"transaction-id": 48087, "amount": 4501}, {"transaction-id": 51930, "amount": 4345}, {"transaction-id": 52633, "amount": 4214}, {"transaction-id": 57717, "amount": 4388}, {"transaction-id": 60042, "amount": 4419}, {"transaction-id": 63538, "amount": 4434}, {"transaction-id": 70414, "amount": 4357}, {"transaction-i

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [8]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Dan',
  'transactions': [{'transaction-id': 14, 'amount': 4526},
   {'transaction-id': 2814, 'amount': 4412},
   {'transaction-id': 4918, 'amount': 4301},
   {'transaction-id': 12573, 'amount': 4215},
   {'transaction-id': 14264, 'amount': 4428},
   {'transaction-id': 15537, 'amount': 4317},
   {'transaction-id': 18212, 'amount': 4644},
   {'transaction-id': 21465, 'amount': 4351},
   {'transaction-id': 24422, 'amount': 4292},
   {'transaction-id': 29525, 'amount': 4297},
   {'transaction-id': 36043, 'amount': 4232},
   {'transaction-id': 41620, 'amount': 4405},
   {'transaction-id': 42443, 'amount': 4356},
   {'transaction-id': 43028, 'amount': 4243},
   {'transaction-id': 46824, 'amount': 4320},
   {'transaction-id': 48087, 'amount': 4501},
   {'transaction-id': 51930, 'amount': 4345},
   {'transaction-id': 52633, 'amount': 4214},
   {'transaction-id': 57717, 'amount': 4388},
   {'transaction-id': 60042, 'amount': 4419},
   {'transaction-id': 63538, 'amount': 44

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [9]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 32,
  'name': 'Alice',
  'transactions': [{'transaction-id': 91, 'amount': 639},
   {'transaction-id': 1961, 'amount': 616},
   {'transaction-id': 3335, 'amount': 625},
   {'transaction-id': 4887, 'amount': 628},
   {'transaction-id': 5255, 'amount': 619},
   {'transaction-id': 6981, 'amount': 590},
   {'transaction-id': 7348, 'amount': 629},
   {'transaction-id': 9342, 'amount': 625},
   {'transaction-id': 9653, 'amount': 623},
   {'transaction-id': 10116, 'amount': 626},
   {'transaction-id': 10185, 'amount': 588},
   {'transaction-id': 10454, 'amount': 603},
   {'transaction-id': 10811, 'amount': 628},
   {'transaction-id': 11162, 'amount': 618},
   {'transaction-id': 12914, 'amount': 647},
   {'transaction-id': 12995, 'amount': 604},
   {'transaction-id': 14087, 'amount': 622},
   {'transaction-id': 15828, 'amount': 622},
   {'transaction-id': 16813, 'amount': 637},
   {'transaction-id': 19480, 'amount': 619},
   {'transaction-id': 21538, 'amount': 598},
   {'transaction-id

In [10]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 122},
 {'name': 'Alice', 'count': 160},
 {'name': 'Alice', 'count': 398},
 {'name': 'Alice', 'count': 310},
 {'name': 'Alice', 'count': 134})

In [11]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(122, 160, 398, 310, 134)

In [12]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

215.52896551724137

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [13]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 91, 'amount': 639},
  {'transaction-id': 1961, 'amount': 616},
  {'transaction-id': 3335, 'amount': 625},
  {'transaction-id': 4887, 'amount': 628},
  {'transaction-id': 5255, 'amount': 619},
  {'transaction-id': 6981, 'amount': 590},
  {'transaction-id': 7348, 'amount': 629},
  {'transaction-id': 9342, 'amount': 625},
  {'transaction-id': 9653, 'amount': 623},
  {'transaction-id': 10116, 'amount': 626},
  {'transaction-id': 10185, 'amount': 588},
  {'transaction-id': 10454, 'amount': 603},
  {'transaction-id': 10811, 'amount': 628},
  {'transaction-id': 11162, 'amount': 618},
  {'transaction-id': 12914, 'amount': 647},
  {'transaction-id': 12995, 'amount': 604},
  {'transaction-id': 14087, 'amount': 622},
  {'transaction-id': 15828, 'amount': 622},
  {'transaction-id': 16813, 'amount': 637},
  {'transaction-id': 19480, 'amount': 619},
  {'transaction-id': 21538, 'amount': 598},
  {'transaction-id': 21560, 'amount': 636},
  {'transaction-id': 21615, 'amount': 612},

In [14]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 91, 'amount': 639},
 {'transaction-id': 1961, 'amount': 616},
 {'transaction-id': 3335, 'amount': 625})

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(639, 616, 625)

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

604.8521392436252

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [17]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [18]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [19]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In other words, ```foldby``` provides a combined groupby and reduce for efficient parallel split-apply-combine tasks. In this case the following cell:

In [22]:
bb.foldby(key, binop, init)

NameError: name 'key' is not defined

is equivalent to these line of code:

In [23]:
def reduction(group):                               
    return reduce(binop, group, init) 

bb.groupby(key).map(lambda (k, v): (k, reduction(v)))

SyntaxError: invalid syntax (<ipython-input-23-b5895395b9d2>, line 4)

In [ ]:
So it allows us to be much faster:

In [21]:
bb = db.from_sequence(range(10))
bb.compute()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [24]:
iseven = lambda x: x % 2 == 0
add = lambda x, y: x + y
dict(bb.foldby(iseven, add))

{True: 20, False: 25}

The results is basically this: ```0+2+4+6+8 = 20``` (even group) vs ```1+3+5+7+9 = 25``` (odd group)

Come back to the previuos example:

In [25]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [26]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 348), ('Alice', 348), ('Alice', 377), ('Alice', 377), ('Bob', 211), ('Bob', 215), ('Bob', 229), ('Bob', 230), ('Charlie', 193), ('Charlie', 196), ('Charlie', 210), ('Charlie', 210), ('Dan', 156), ('Dan', 156), ('Dan', 169), ('Dan', 169), ('Edith', 288), ('Edith', 288), ('Edith', 312), ('Edith', 312), ('Frank', 216), ('Frank', 216), ('Frank', 234), ('Frank', 234), ('George', 180), ('George', 180), ('George', 195), ('George', 195), ('Hannah', 264), ('Hannah', 264), ('Hannah', 286), ('Hannah', 286), ('Ingrid', 240), ('Ingrid', 260), ('Ingrid', 499), ('Jerry', 259), ('Jerry', 263), ('Jerry', 283), ('Jerry', 284), ('Kevin', 274), ('Kevin', 274), ('Kevin', 297), ('Kevin', 299), ('Laura', 168), ('Laura', 168), ('Laura', 182), ('Laura', 182), ('Michael', 312), ('Michael', 312), ('Michael', 336), ('Michael', 338), ('Norbert', 189), ('Norbert', 190), ('Norbert', 205), ('Norbert', 207), ('Oliver', 284), ('Oliver', 287), ('Oliver', 309), ('Oliver', 310), ('Patricia', 204), ('Patricia', 

In [27]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1450), ('Bob', 885), ('Charlie', 809), ('Dan', 650), ('Edith', 1200), ('Frank', 900), ('George', 750), ('Hannah', 1100), ('Ingrid', 999), ('Jerry', 1089), ('Kevin', 1144), ('Laura', 700), ('Michael', 1298), ('Norbert', 791), ('Oliver', 1190), ('Patricia', 850), ('Quinn', 800), ('Ray', 755), ('Sarah', 1050), ('Tim', 899), ('Ursula', 993), ('Victor', 1000), ('Wendy', 808), ('Xavier', 750), ('Yvonne', 750), ('Zelda', 1144)]
CPU times: user 174 ms, sys: 12.7 ms, total: 187 ms
Wall time: 1.35 s


Take a look to the API in order to have a clear vision about foldby: [https://docs.dask.org/en/latest/bag-api.html#dask.bag.Bag.foldby](https://docs.dask.org/en/latest/bag-api.html#dask.bag.Bag.foldby)

### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [ ]:
# Create a cluster between you by using ngrok

In [33]:
# Your code here...
client.close()

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [28]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Dan,"[{'transaction-id': 14, 'amount': 4526}, {'tra..."
1,1,Frank,"[{'transaction-id': 1297, 'amount': 1662}, {'t..."
2,2,George,"[{'transaction-id': 42, 'amount': 526}, {'tran..."
3,3,Wendy,"[{'transaction-id': 1834, 'amount': 4259}, {'t..."
4,4,Michael,"[{'transaction-id': 1030, 'amount': 181}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [29]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 229 ms, sys: 39 ms, total: 268 ms
Wall time: 1.97 s


name
Alice      1450
Bob         885
Charlie     809
Dan         650
Edith      1200
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [30]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Dan', 'amount': 4526, 'transaction-id': 14},
 {'id': 0, 'name': 'Dan', 'amount': 4412, 'transaction-id': 2814},
 {'id': 0, 'name': 'Dan', 'amount': 4301, 'transaction-id': 4918})

In [31]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Dan,4526,14
1,0,Dan,4412,2814
2,0,Dan,4301,4918
3,0,Dan,4215,12573
4,0,Dan,4428,14264


In [32]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 318 ms, sys: 23.4 ms, total: 341 ms
Wall time: 3.55 s


name
Alice       312517
Bob         220521
Charlie     109668
Dan         176768
Edith       222886
Frank       252987
George      130603
Hannah      228005
Ingrid      198532
Jerry       193299
Kevin       295563
Laura       131951
Michael     236330
Norbert     141037
Oliver      219735
Patricia    176694
Quinn       199894
Ray         115767
Sarah       231921
Tim         169455
Ursula      109279
Victor      208168
Wendy       205532
Xavier      136459
Yvonne      157207
Zelda       219222
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.